In [4]:
from cosecha_colectiva import miscelaneous as m
m.try_find_conf_file()

import datajoint as dj

from cosecha_colectiva import main_tester as mt
from cosecha_colectiva import query_cosecha as qc
from cosecha_colectiva import config
from cosecha_colectiva import CAF_DB_Monitor as CDM
from cosecha_colectiva import test_cosecha_api as tca

import logging
import pathlib
from datetime import datetime
import pandas as pd

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [5]:
#xls_name = 'CAF_GrupoTesteador.xlsx'
xls_name = 'CAF_GrupoAstronomo.xlsx'
xls_name = 'CAF_GrupoMesero.xlsx'
#xls_name = 'CAF_GrupoDomador.xlsx'
#xls_name = 'CAF_GrupoCafam.xlsx'
#xls_name = 'GrupoCafam.xlsx'
#xls_name = 'CAF_GrupoHidalgo.xlsx'
#xls_name = 'CAF_GrupoCalcetin.xlsx'
#xls_name = 'Alcancía Viva'

In [6]:
cosecha_db = dj.create_virtual_module(config.db_name, config.db_name)


In [12]:
grupo_q = dict()
grupo_q['Grupo_id'] = 2394
grupo = (cosecha_db.Grupos & grupo_q).fetch('KEY', as_dict=True)
grupo

[{'Grupo_id': 2394}]

In [13]:
id_grupo = qc.get_grupo_id_by_name(xls_name.split('.')[0])
print(id_grupo)

[2394, True]


In [14]:
id_grupo

[2394, True]

In [16]:
qc.delete_grupo(id_grupo[0], solo_sesiones=True, force_delete=False)

([{'Transaccion_prestamo_id': 10751}, {'Transaccion_prestamo_id': 10836}, {'Transaccion_prestamo_id': 10845}, {'Transaccion_prestamo_id': 10854}, {'Transaccion_prestamo_id': 10866}, {'Transaccion_prestamo_id': 10867}, {'Transaccion_prestamo_id': 10885}, {'Transaccion_prestamo_id': 10886}], 'transaccion_prestamos')
DELETE from railway.transaccion_prestamos WHERE Transaccion_prestamo_id in (10751, 10836, 10845, 10854, 10866, 10867, 10885, 10886)
([{'Transaccion_id': 37820}, {'Transaccion_id': 37821}, {'Transaccion_id': 37822}, {'Transaccion_id': 37832}, {'Transaccion_id': 37836}, {'Transaccion_id': 37837}, {'Transaccion_id': 37838}, {'Transaccion_id': 37839}, {'Transaccion_id': 37840}, {'Transaccion_id': 37841}, {'Transaccion_id': 37842}, {'Transaccion_id': 37843}, {'Transaccion_id': 39040}, {'Transaccion_id': 39041}, {'Transaccion_id': 39042}, {'Transaccion_id': 39043}, {'Transaccion_id': 39044}, {'Transaccion_id': 39045}, {'Transaccion_id': 39046}, {'Transaccion_id': 39047}, {'Transacc

In [17]:
now = datetime.now()
log_date = now.strftime("%Y-%m-%d_%H_%M_%S")
log_file = pathlib.Path(config.log_dir, log_date+'_Sesiones.log')
log_file
logging.basicConfig(filename=log_file, encoding='utf-8', level=logging.INFO)

In [18]:
# tca.CAF_API_sessions_tester.poner_corriente_grupo(xls_name)

In [21]:
monitor_grupo = CDM.CAF_DB_Monitor(id_grupo[0], xls_name)

In [22]:
monitor_grupo.limite_credito_socios

,Grupo_socio_id,Socio_id,Tipo_socio,Status,Acciones,Grupo_id,CURP,xls_order,monto_total,num_prestamos,Limite_credito_final,Limite_credito_teorico,puede_pedir_prestamo
0,8134,1194,ADMIN,1,100,2394,MEST770222MMSJNR01,0,0,0,2000,2000,True
1,8144,1204,SUPLENTE,1,100,2394,MEST770222MMSJNR02,1,0,0,2000,2000,True
2,8154,1214,SOCIO,1,100,2394,MEST770222MMSJNR03,2,0,0,2000,2000,True
3,8164,1224,SOCIO,1,100,2394,MEST770222MMSJNR04,3,0,0,2000,2000,True
4,8174,1234,SOCIO,1,100,2394,MEST770222MMSJNR05,4,0,0,2000,2000,True
5,8184,1244,SOCIO,1,100,2394,MEST770222MMSJNR06,5,0,0,2000,2000,True
6,8194,1254,SOCIO,1,100,2394,MEST770222MMSJNR07,6,0,0,2000,2000,True
7,8204,1264,SOCIO,1,100,2394,MEST770222MMSJNR08,7,0,0,2000,2000,True
8,8214,1274,SOCIO,1,100,2394,MEST770222MMSJNR09,8,0,0,2000,2000,True
9,8224,1284,SOCIO,1,100,2394,MEST770222MMSJNR10,9,0,0,2000,2000,True


In [23]:
start_month = 1
end_month = 1

for month in range(start_month, end_month+1):

    print('month: ', month)

    try:
        monitor_grupo.actualiza_todo_sesion(xls_name, month, type_xls='NEW')
    except Exception as e:
        print(e)
        raise(e)
    try:
        mt.test_sesion_months(xls_name, month, month, type_xls='NEW', id_grupo=id_grupo)
    except Exception as e:
        print(e)
        raise(e)
    [status, dfs] = monitor_grupo.compara_db_monitor()
    if not status:
        print('Algo no coincidió')
        break

month:  1
<Response [503]>
Login incorrecto


Exception: Login incorrecto

In [12]:
monitor_grupo.ganancias_sesion

,Acciones,Socio_id,Monto_ganancia_floor,sobrante_Monto_ganancia,Monto_ganancia,Entregada,Sesion_id,Ganancias_id
0,100,1194,0.5,0.5,1.0,0,19492,78684
1,100,1204,0.5,0.5,1.0,0,19492,78685
2,100,1214,0.5,0.5,1.0,0,19492,78686
3,100,1224,0.5,0.5,1.0,0,19492,78687
4,100,1234,0.5,0.5,1.0,0,19492,78688
5,100,1244,0.5,0.5,1.0,0,19492,78689
6,100,1254,0.5,0.0,0.5,0,19492,78690
7,100,1264,0.5,0.0,0.5,0,19492,78691
8,100,1274,0.5,0.0,0.5,0,19492,78692
9,100,1284,0.5,0.0,0.5,0,19492,78693


In [13]:
monitor_grupo.prestamos.loc[0,:]

Prestamo_id                      6169
Monto_prestamo                  500.0
Monto_pagado                     29.0
Interes_generado                 39.5
Interes_pagado                   30.0
Estatus_ampliacion              False
Num_sesiones                        6
Sesiones_restantes                  2
Estatus_prestamo                    0
Socio_id                         1234
Sesion_id                       19488
Acuerdos_id                      2044
Prestamo_original_id             None
Grupo_id                         2394
Tasa_interes                      2.0
Limite_credito                     20
Creditos_simultaneos                2
Interes_morosidad                 1.5
Ampliacion_prestamos                1
Interes_ampliacion                1.5
Mod_calculo_interes                 0
Tasa_interes_prestamo_grande      1.5
interes_futuro                    9.5
Ultimo_interes_pagado               0
debe_interes                      9.5
sobrante_abono                    NaN
Ultimo_abono

In [14]:
monitor_grupo.interes_prestamo

,Interes_prestamo_id,Prestamo_id,Sesion_id,Monto_interes,Tipo_interes
0,103148,6169,19489,10.0,0
1,103149,6170,19490,10.0,0
2,103150,6169,19490,10.0,0
3,103151,6170,19491,10.0,0
4,103152,6169,19491,10.0,0
5,103153,6170,19492,8.0,0
6,103154,6169,19492,9.5,0


In [15]:
dfs

[]

In [19]:
monitor_grupo.ganancias.groupby('Socio_id').sum()

,Ganancias_id,Monto_ganancia,Entregada,Sesion_id
Socio_id,,,,
1194,461494,6.5,1,113394
1204,461509,6.5,1,113394
1214,461524,6.5,1,113394
1224,461539,6.5,1,113394
1234,461554,6.0,1,113394
1244,461569,6.0,1,113394
1254,461584,5.5,1,113394
1264,461599,5.5,1,113394
1274,461614,4.5,1,113394


In [16]:
monitor_grupo.sesiones

[{'Sesion_id': 15944},
 {'Sesion_id': 19488},
 {'Sesion_id': 19489},
 {'Sesion_id': 19490},
 {'Sesion_id': 19491},
 {'Sesion_id': 19492}]

In [22]:
118.75+9.5

128.25

In [26]:
161.5-9.5

152.0

In [27]:
475/152

3.125

In [25]:
475/4+9.5

128.25

In [12]:
monitor_grupo.socios_acciones.loc[monitor_grupo.socios_acciones['Socio_id'] == 1324, 'Status'].values[0]

1

In [10]:
comp = dfs[0].compare(dfs[1])
comp

Monto_ganancia          
             self     other
24           1.25  1.428571
25           1.25  1.428571
26           1.25  1.428571
27           1.25  1.428571
28           1.25  1.428571
29           1.25  1.428571
30           1.25  1.428571

In [19]:
monitor_grupo.ganancias

,Ganancias_id,Monto_ganancia,Entregada,Socio_id,Sesion_id
0,49264,0.000000,0,1324,6994
1,49274,0.000000,0,1334,6994
2,49284,0.000000,0,1344,6994
3,49294,0.000000,0,1354,6994
4,49304,0.000000,0,1364,6994
5,49314,0.000000,0,1374,6994
6,49324,0.000000,0,1384,6994
7,49334,0.000000,0,1394,6994
8,52324,0.000000,0,1324,7434
9,52334,0.000000,0,1334,7434


In [ ]:
monitor_grupo

In [18]:
monitor_grupo.multas

,Multa_id,Monto_multa,Descripcion,Status,Sesion_id,Socio_id,Pago_en_sesion
0,1524,10.0,Multa # 1 socio 1534 grupo: 1574,1,7114,1534,0
1,1534,10.0,Multa # 1 socio 1584 grupo: 1574,1,7114,1584,0
2,1544,20.0,Multa # 1 socio 1614 grupo: 1574,1,7134,1614,0


In [22]:
monitor_grupo.prestamos

,Prestamo_id,Monto_prestamo,Monto_pagado,Interes_generado,Interes_pagado,Estatus_ampliacion,Num_sesiones,Sesiones_restantes,Estatus_prestamo,Socio_id,...,Interes_morosidad,Ampliacion_prestamos,Interes_ampliacion,Mod_calculo_interes,Tasa_interes_prestamo_grande,interes_futuro,Ultimo_interes_pagado,debe_interes,sobrante_abono,Ultimo_abono
0,2894,1000.0,1000.0,60.0,60.0,False,6,3,1,1574,...,0.0,0,None,1,1.5,0.0,0,0,0,0
1,2904,600.0,400.0,36.0,24.0,False,6,3,0,1524,...,0.0,0,None,1,1.5,12.0,12.0,12.0,200.0,212.0
2,2914,590.0,400.4,35.4,23.6,False,6,3,0,1554,...,0.0,0,None,1,1.5,11.8,11.8,11.8,200.2,212.0
3,2924,590.0,400.4,35.4,23.6,False,6,3,0,1604,...,0.0,0,None,1,1.5,11.8,11.8,11.8,200.2,212.0
4,2934,675.0,225.5,27.0,13.5,False,6,4,0,1544,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
5,2944,675.0,225.5,27.0,13.5,False,6,4,0,1564,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
6,2954,675.0,225.5,27.0,13.5,False,6,4,0,1584,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
7,2964,675.0,225.5,27.0,13.5,False,6,4,0,1594,...,0.0,0,None,1,1.5,13.5,13.5,13.5,225.5,239.0
8,2974,1000.0,0,20.0,0,False,6,5,0,1534,...,0.0,0,None,1,1.5,20.0,0,0,0,0
9,2984,1200.0,0,24.0,0,False,6,5,0,1574,...,0.0,0,None,1,1.5,24.0,0,0,0,0


In [19]:
monitor_grupo.ganancias

,Ganancias_id,Monto_ganancia,Entregada,Socio_id,Sesion_id
0,49504,0.000000,0,1524,7034
1,49514,0.000000,0,1534,7034
2,49524,0.000000,0,1544,7034
3,49534,0.000000,0,1554,7034
4,49544,0.000000,0,1564,7034
5,49554,0.000000,0,1574,7034
6,49564,0.000000,0,1584,7034
7,49574,0.000000,0,1594,7034
8,49584,0.000000,0,1604,7034
9,49594,0.000000,0,1614,7034


In [13]:
monitor_grupo.sesiones

[{'Sesion_id': 2174},
 {'Sesion_id': 6224},
 {'Sesion_id': 6234},
 {'Sesion_id': 6244},
 {'Sesion_id': 6254},
 {'Sesion_id': 6264},
 {'Sesion_id': 6274},
 {'Sesion_id': 6284},
 {'Sesion_id': 6294},
 {'Sesion_id': 6304}]

In [21]:
monitor_grupo.ganancias.groupby('Sesion_id').sum()

,Ganancias_id,Monto_ganancia,Entregada,Socio_id
Sesion_id,,,,
6994,394392,0.0,0,10872
7434,418872,0.0,0,10872
7444,419512,10.0,0,10872
7454,367598,10.0,0,9488
